# Imágenes satelitales NASA SpaceApps training [@AESS Unicauca](https://www.facebook.com/aess.unicauca)


<div>
<img src="https://scontent.fppn1-1.fna.fbcdn.net/v/t39.30808-6/296685322_579745683865854_8669697208972745235_n.jpg?_nc_cat=104&ccb=1-7&_nc_sid=09cbfe&_nc_ohc=ik1-U666nBMAX-Ti4lm&_nc_oc=AQl7zqSsW1aWRJgr212b3jEe7EVZwKuo-NjdP14Ku67Wb02KMD2pW9Yp1SMn7wzpScs&_nc_ht=scontent.fppn1-1.fna&oh=00_AT_AmLQyPM-cV1DG3nITWRevSgS2lys7t-gNvSihmHJV1w&oe=633AB925" width="700" height = "400"/>
</div>

Ejemplo práctico con Tensorflow. 

Por [Sebastian Cajas Ordóñez](https://sebasmos.github.io/)  

## Outline
1. [Deep Learning - Ejemplo práctico](#10)

<a name='9'></a>
# Practical example - Deep learning

Para instalación en entorno local

# Requires the latest pip
`!pip install --upgrade pip`

# Current stable release for CPU and GPU
`!pip install tensorflow`

# Or try the preview build (unstable)
`!pip install tf-nightly`


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

In [65]:
df = pd.read_csv("weatherHistory.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Formatted Date            96453 non-null  object 
 1   Summary                   96453 non-null  object 
 2   Precip Type               95936 non-null  object 
 3   Temperature (C)           96453 non-null  float64
 4   Apparent Temperature (C)  96453 non-null  float64
 5   Humidity                  96453 non-null  float64
 6   Wind Speed (km/h)         96453 non-null  float64
 7   Wind Bearing (degrees)    96453 non-null  float64
 8   Visibility (km)           96453 non-null  float64
 9   Loud Cover                96453 non-null  float64
 10  Pressure (millibars)      96453 non-null  float64
 11  Daily Summary             96453 non-null  object 
dtypes: float64(8), object(4)
memory usage: 8.8+ MB


In [66]:
def process(df):
  df['Precip Type'].fillna(method='ffill',inplace=True,axis=0)
  df['Precip Type'].value_counts()
  #with droping
  df.drop('Precip Type',axis=1,inplace=True)
  target_values=df['Summary'].value_counts().index
  le=LabelEncoder()
  df['Summary']=le.fit_transform(df['Summary'])
  print("There are {} classes.".format(len(np.unique(df["Summary"]))))
  y=df['Summary']
  x=df[['Temperature (C)',
        'Apparent Temperature (C)', 'Humidity', 'Wind Speed (km/h)',
        'Wind Bearing (degrees)', 'Visibility (km)',
        'Pressure (millibars)']]
  x.drop('Apparent Temperature (C)',axis=1,inplace=True)
  return x,y
x,y = process(df)

There are 27 classes.


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [67]:
n_classes = len(np.unique(y))

labels_cat = to_categorical(y, num_classes=n_classes)

In [68]:
x_train,x_test,y_train,y_test=train_test_split(x, labels_cat,test_size=0.2,random_state=1)

print("Training instances   {}, Training features   {}".format(x_train.shape[0], x_train.shape[1]))
print("Testing instances    {}, Testing features    {}".format(x_test.shape[0], x_test.shape[1]))
print("Hot-encoded label: ", y_train.shape)

Training instances   77162, Training features   6
Testing instances    19291, Testing features    6
Hot-encoded label:  (77162, 27)


In [15]:
weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(np.ravel(y,order='C')),
                                        y = np.ravel(y,order='C')                                                    
                                    )

In [81]:
def get_model_baseline(x,n_classes):
  model_keras_seq = Sequential()
  model_keras_seq.add(Dense(128, input_shape=(x.shape), activation='relu'))
  model_keras_seq.add(Dense(64, activation='relu'))
  model_keras_seq.add(Dense(n_classes, activation='sigmoid'))
  return model_keras_seq

In [82]:
model = get_model_baseline(x_train, n_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 77162, 128)        896       
                                                                 
 dense_24 (Dense)            (None, 77162, 64)         8256      
                                                                 
 dense_25 (Dense)            (None, 77162, 27)         1755      
                                                                 
Total params: 10,907
Trainable params: 10,907
Non-trainable params: 0
_________________________________________________________________


In [83]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,TensorBoard

earlystopper = EarlyStopping(monitor='val_acc',
                             patience=50,
                             mode='max',
                             restore_best_weights=True)

tensorboard = TensorBoard(log_dir='./logs', write_graph=True,
                          write_images=True, update_freq='epoch')

In [86]:
y_train.shape

(77162, 27)

In [ ]:
example = model.fit(x_train, y_train, 
                    batch_size = 64, 
                    verbose=1, 
                    epochs=25, 
                    validation_split = 0.2,
                    shuffle=False,
                    #class_weight=weights,
                    callbacks=[earlystopper, tensorboard])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
_, acc = model.evaluate(x_test, y_test)
print("Accuracy is = ", (acc * 100.0), "%")